In [0]:
import datetime

In [0]:
class Scheduler:
    """ Classe que defini quando as tarefas serão executadas. """

    def __init__(self, function, start_date: str, nome='', duration=1, trigger_interval=1, skip_fisrt=False):
        """
        ARGUMENTOS
        ----------
        duration : int
            Duração da execução em DIAS
        trigger_interval : int
            Intervalo entre as execuções em HORAS
        """

        self.nome = nome
        self.function = function
        # 2023-09-26 to datetime
        self.start_date = datetime.datetime.fromisoformat(start_date)
        # Dias de processamento
        self.duration = datetime.timedelta(days=duration)
        # Intervalo entre execuções
        self.trigger_interval = datetime.timedelta(hours=trigger_interval)
        self.skip_fisrt = skip_fisrt

        # Calcula o dia de fim da execução
        self.end_date = self.start_date + self.duration

    @property
    def agenda(self) -> list:
        """ Lista de tuplas com o horário e a função da tarefa a ser executada. 

        RETURN
        ------
            List[ Tuple[ datetime, function ] ]
                A lista segue a estrutura de PILHA. Os últimos itens são as próximas execuções.
        """
        agenda_list = []
        current_time = self.start_date

        # Pula a primeira execução
        if self.skip_fisrt:
            current_time = current_time + self.trigger_interval

        # Obtenção de todos os horários até a o fim da execução
        while current_time < self.end_date:
            agenda_list.append( (current_time, self.function) )
            current_time = current_time + self.trigger_interval

        return agenda_list[::-1]
    


class Executor:
    """ Classe de execução do timer e ativação das tarefas agendas. 
    
    EXAMPLE
    -------
        execs = Executor(tarefa_hourly, tarefa_monthly)
        execs.run()
    """
    def __init__(self, *args: list):
        # Lista de agendamentos das tarefas
        self.agendas = [tarefa.agenda for tarefa in args]
        self.tarefas = args

    def run(self):
        # Inicia a execução contínua do Executor
        print('Iniciando execução contínua das tarefas.')
        inicio_execucao = datetime.datetime.now()
        proximo_minuto = inicio_execucao + datetime.timedelta(minutes=1)

        agendas = []
        for task in self.agendas:
            agendas.extend(task)
        agendas = sorted(agendas, reverse=True, key=lambda x: x[0])

        while True:
            # Granularidade da execução em 1 segundo
            time.sleep(1)
            now = datetime.datetime.now()

            # Sinalização de execução
            if now > proximo_minuto:
                print(now)
                proximo_minuto = now + datetime.timedelta(minutes=1)

            
            # Obtenção da próxima execução
            if len(agendas) > 0:
                proxima_execucao = agendas[-1][0]

                # Trigger
                if proxima_execucao < now:
                    print(f'Tarefa agendada às [{proxima_execucao}] em execução. Ativação às {now}')

                    # Execução a tarefa
                    tarefa = agendas.pop()
                    task_function = tarefa[1]
                    task_function(proxima_execucao)
                    print('Função executada. Scheduler em espera.......\n')

            if len(agendas) == 0:
                print('Todas as tarefas executadas! Fim do Executor.')
                break

In [0]:
def printar(valor):
    print(valor)